We receive a .txt file from a vendor, but the content is comma-separated values. How do you process it?”

Solution 1: Read using Python csv module (BEST PRACTICE)

In [3]:
import csv

with open("orders.txt", mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)

    for row in reader:
        print(row)


{'order_id': '101', 'customer': 'Alice', 'amount': '250', 'order_date': '2024-01-01'}
{'order_id': '102', 'customer': 'Bob', 'amount': '400', 'order_date': '2024-01-02'}
{'order_id': '103', 'customer': 'Charlie', 'amount': '150', 'order_date': '2024-01-03'}


Solution 2: Using pandas (Most common in DE)

In [6]:
import pandas as pd

df = pd.read_csv("orders.txt")
print(df)


   order_id customer  amount  order_date
0       101    Alice     250  2024-01-01
1       102      Bob     400  2024-01-02
2       103  Charlie     150  2024-01-03


Level 2 — File extension is wrong AND delimiter is weird

Answer: Delimiter mismatch is common; I inspect sample rows and set delimiter explicitly.”

In [7]:
import pandas as pd

df = pd.read_csv("transactions.txt", delimiter="|")
print(df)


   txn_id user  amount   status
0       1    A     500  success
1       2    B     300   failed


Level 3 — File has extra junk lines (VERY COMMON)

Scenario

Vendor adds metadata before actual data

Solution: Skip rows

In [12]:
import pandas as pd

df = pd.read_csv("sales.txt", skiprows=3)
print(df)


   order_id  amount
0         1     100
1         2     200



🔹 Level 4 — File has NO extension at all


Scenario

Data comes as data_dump (no .csv, no .txt)

Detect content automatically

In [16]:
with open("data_dump", "r") as f:
    first_line = f.readline()

print(first_line)


if "," in first_line:
    print("Looks like CSV")
elif "|" in first_line:
    print("Looks like pipe delimited")


txn_id|user|amount|status

Looks like pipe delimited


Level 5 — CSV inside .txt but values have commas (🔥 tricky)

Correct handling

In [17]:
import pandas as pd

df = pd.read_csv("customers.txt", quotechar='"')
print(df)


   id   name             address
0   1  Alice  Street 1, New York
1   2    Bob    Street 2, London


Level 6 — Mixed bad rows (real production pain

Scenario

Some rows are corrupted

In [ ]:
# Handle safely
# the file is like 
'''
id,amount
1,100
2,ABC
3,
4,,    ------->badline will be skiped/warned/errored
3,300

'''

import pandas as pd

df = pd.read_csv("sales_with_badlines.txt", on_bad_lines="skip")
print(df)


   id amount
0   1    100
1   2    ABC
2   3    NaN
3   3    300


PART 2 — Read file safely using CHUNKS (Production way)

Interview scenario

“How do you process a large file with bad rows without crashing?”

In [33]:
import pandas as pd

chunk_size = 1000
valid_rows = []

for chunk in pd.read_csv(
    "large_orders.txt",
    chunksize=chunk_size,
    on_bad_lines="skip"
    # on_bad_lines="error"
    # on_bad_lines="warn"
):
    # Convert amount safely
    chunk["amount"] = pd.to_numeric(chunk["amount"], errors="coerce")

    # Drop rows where amount is invalid
    clean_chunk = chunk.dropna(subset=["amount"])

    valid_rows.append(clean_chunk)
    # valid_rows.append(chunk)

final_df = pd.concat(valid_rows, ignore_index=True)

print(final_df.shape)
print(final_df.head())


(9979, 4)
  order_id customer  amount  order_date
0        1    David   581.0  2024-07-12
1        2  Charlie   658.0  2024-09-08
2        3    David   547.0  2024-01-29
3        4    Alice   723.0  2024-12-30
4        5    David   677.0  2024-10-16


PART 3 — Log bad rows separately (VERY IMPRESSIVE)

In [34]:
bad_rows = []

for chunk in pd.read_csv(
    "large_orders.txt",
    chunksize=1000,
    on_bad_lines="skip"
):
    chunk["amount"] = pd.to_numeric(chunk["amount"], errors="coerce")

    invalid = chunk[chunk["amount"].isna()]
    bad_rows.append(invalid)

bad_df = pd.concat(bad_rows, ignore_index=True)
bad_df.to_csv("bad_rows.csv", index=False)

print("Bad rows saved:", bad_df.shape)


Bad rows saved: (21, 4)


PART 4 — Edge Scenarios

File suddenly changes schema (column added)

In [35]:
expected_cols = {"order_id", "customer", "amount", "order_date"}

for chunk in pd.read_csv("large_orders.txt", chunksize=1000):
    if set(chunk.columns) != expected_cols:
        raise ValueError("Schema mismatch detected")


In [37]:
import os

if os.path.getsize("empty.txt") == 0:
    print("File is empty, skipping processing")


File is empty, skipping processing


In [38]:
df = final_df.drop_duplicates(subset=["order_id"])


In [40]:
with open("large_orders.txt", "rb") as f:
    content = f.read().replace(b"\x00", b"")

with open("cleaned.txt", "wb") as f:
    f.write(content)
